# MultiOn Step Tracing
<img src="https://github.com/AgentOps-AI/agentops/blob/b4aac2d4b9fb16d6aa0a25aa9018210a94f1bef2/docs/logo/multion_integration.png?raw=true" width="250px" style="max-width: 100%; height: auto;"/>
    
With AgentOps, agent actions as well as MultiOn browse events will get tracked. MultiOn browse events automatically trace screenshots as well.

This example shows how to use MultiOn's session creator to launch a self-directed browser agent that accomplishes a specified objective using Step Mode. MultiOn agents can either accomplish tasks fully autonomously or managed one step at a time. In this example, we will launch a MutliOn agent and manage each individual step. 

First let's install the required packages

In [ ]:
%pip install -U multion
%pip install -U agentops
%pip install -U python-dotenv

Then import them

In [ ]:
import multion
from multion.client import MultiOn
from multion.sessions.types.sessions_step_request_browser_params import (
    SessionsStepRequestBrowserParams,
)
from multion.core.request_options import RequestOptions
import agentops
import os
from dotenv import load_dotenv

Next, we'll set our API keys. There are several ways to do this, the code below is just the most foolproof way for the purposes of this notebook. It accounts for both users who use environment variables and those who just want to set the API Key here in this notebook.

[Get an AgentOps API key](https://agentops.ai/settings/projects)

1. Create an environment variable in a .env file or other method. By default, the AgentOps `init()` function will look for an environment variable named `AGENTOPS_API_KEY`. Or...

2. Replace `<your_agentops_key>` below and pass in the optional `api_key` parameter to the AgentOps `init(api_key=...)` function. Remember not to commit your API key to a public repo!

In [ ]:
load_dotenv()
MULTION_API_KEY = os.getenv("MULTION_API_KEY") or "<your_multion_key>"
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY") or "<your_agentops_key>"

# Session streaming
In this example, we'll use MultiOn to stream individual steps to accomplish a task. To track your runs in the AgentOps dashboard, specify an `agentops_api_key` when initializing `MultiOn()`

You can run MultiOn without running `agentops.init()`. However, you will only see events from MultiOn, and not any from your own agent.


In [ ]:
multion = MultiOn(
    api_key=MULTION_API_KEY,
    agentops_api_key=AGENTOPS_API_KEY,
)

url = "https://www.agentops.ai/"
cmd = "what three things do i get with agentops"
create_session_response = multion.sessions.create(url=url)
session_id = create_session_response.session_id
print(create_session_response.message)
browser_params = SessionsStepRequestBrowserParams(height=1.1, width=1.1)
request_options = RequestOptions(
    timeout_in_seconds=60, max_retries=2, additional_headers={"test": "ing"}
)
step_session_response = multion.sessions.step(
    session_id=session_id,
    cmd=cmd,
    url=url,
    browser_params=browser_params,
    include_screenshot=True,
    request_options=request_options,
)
print(step_session_response.message)

while step_session_response.status == "CONTINUE":
    step_session_response = multion.sessions.step(
        session_id=session_id,
        cmd=cmd,
        url=url,
        include_screenshot=True,
    )
    print(step_session_response.message)

if step_session_response.status == "DONE":
    print("task completed")

get_screenshot = multion.sessions.screenshot(session_id=session_id)
close_session_response = multion.sessions.close(session_id=session_id)

# Step Stream
Step stream is just like step, but it streams responses in the same way a streamed LLM response is received. Instead of waiting for the entire step to complete, MultiOn will return work in progress. To track your runs in the AgentOps dashboard, specify an `agentops_api_key` when initializing `MultiOn()`

In [ ]:
import multion
from multion.client import MultiOn
from multion.sessions.types.sessions_step_stream_request_browser_params import (
    SessionsStepStreamRequestBrowserParams,
)
from multion.core.request_options import RequestOptions
import os

multion = MultiOn(
    api_key=MULTION_API_KEY,
    agentops_api_key=AGENTOPS_API_KEY,
)

url = "https://www.agentops.ai/"
cmd = "what three things do i get with agentops"
create_session_response = multion.sessions.create(url=url)
session_id = create_session_response.session_id
print(create_session_response.message)
browser_params = SessionsStepStreamRequestBrowserParams(height=1.1, width=1.1)
request_options = RequestOptions(
    timeout_in_seconds=60, max_retries=2, additional_headers={"test": "ing"}
)
step_session_response = multion.sessions.step_stream(
    session_id=session_id,
    cmd=cmd,
    url=url,
    browser_params=browser_params,
    include_screenshot=True,
    request_options=request_options,
)

for chunk in step_session_response:
    if chunk.type == "final_event":
        break
    print(chunk.delta["content"])

while chunk.delta["status"] == "CONTINUE":
    step_session_response = multion.sessions.step_stream(
        session_id=session_id,
        cmd=cmd,
        url=url,
        include_screenshot=True,
    )
    for chunk in step_session_response:
        if chunk.type == "final_event":
            break
        print(chunk.delta["content"])

if chunk.delta["status"] == "DONE":
    print("task completed")

get_screenshot = multion.sessions.screenshot(session_id=session_id)
close_session_response = multion.sessions.close(session_id=session_id)

## Check your session
Check your session on [AgentOps](https://app.agentops.ai)

![AgentOps MultiOn Step](https://github.com/AgentOps-AI/agentops/blob/main/docs/images/agentops-multion-step.gif?raw=true)

![AgentOps Session Overview](https://github.com/AgentOps-AI/agentops/blob/main/docs/images/agentops-session-overview.png?raw=true)